In [15]:

import pandas as pd
import numpy as np
import os

# # Set the working directory
# os.chdir("1-27-26_exercise/Untitled.ipynb")

# # # Read the .sav file
df1 = pd.read_spss("USA1975withspousehfep.sav")
df2 = pd.read_spss("USA1975withspousehfsum.sav")

# # # Print the dataframe
print(df1,df2)


# # # Write the .csv file
# df.to_csv("file.csv")


                               survey                      wave    hhid  \
0       1975-1976 longitudinal survey   first wave or only wave     1.0   
1       1975-1976 longitudinal survey   first wave or only wave     1.0   
2       1975-1976 longitudinal survey   first wave or only wave     1.0   
3       1975-1976 longitudinal survey   first wave or only wave     1.0   
4       1975-1976 longitudinal survey   first wave or only wave     1.0   
...                               ...                       ...     ...   
158242  1975-1976 longitudinal survey  4th wave / 4th year ATUS  1516.0   
158243  1975-1976 longitudinal survey  4th wave / 4th year ATUS  1516.0   
158244  1975-1976 longitudinal survey  4th wave / 4th year ATUS  1516.0   
158245  1975-1976 longitudinal survey  4th wave / 4th year ATUS  1516.0   
158246  1975-1976 longitudinal survey  4th wave / 4th year ATUS  1516.0   

           pid                 spouse  couple diaryday  cday      month  \
0          1.0  main res